**Class Definition and Initialization**

This section defines the class TspDynamicProgrammingRecursive and initializes its fields. It includes validation checks ensure the input matrix is square and the starting node is within valid bounds.

In [11]:
class TspDynamicProgrammingRecursive:
  def __init__(self, distance, start_node=0):
        self.distance = distance
        self.N = len(distance)
        self.START_NODE = start_node
        self.FINISHED_STATE = (1 << self.N) - 1
        self.minTourCost = float('inf')
        self.tour = []
        self.ranSolver = False

        # Validate inputs
        if self.N <= 2:
            raise ValueError("TSP on 0, 1 or 2 nodes doesn't make sense.")
        if self.N != len(distance[0]):
            raise ValueError("Matrix must be square (N x N)")
        if self.START_NODE < 0 or self.START_NODE >= self.N:
            raise ValueError("Starting node must be: 0 <= start_node < N")
        if self.N > 32:
            raise ValueError("Matrix too large! A matrix that size for the DP TSP problem with a time complexity of O(n^2*2^n) requires way too much computation for any modern home computer to handle")

  def tsp(self, i, state, memo, prev):
        if state == self.FINISHED_STATE:
            return self.distance[i][self.START_NODE]
        if memo[i][state] is not None:
            return memo[i][state]

        min_cost = float('inf')
        index = -1
        for next in range(self.N):
            if state & (1 << next):
                continue
            next_state = state | (1 << next)
            new_cost = self.distance[i][next] + self.tsp(next, next_state, memo, prev)
            if new_cost < min_cost:
                min_cost = new_cost
                index = next

        prev[i][state] = index
        memo[i][state] = min_cost
        return min_cost

  def solve(self):
    state = 1 << self.START_NODE
    memo = [[None] * (1 << self.N) for _ in range(self.N)]
    prev = [[None] * (1 << self.N) for _ in range(self.N)]
    self.minTourCost = self.tsp(self.START_NODE, state, memo, prev)

    index = self.START_NODE
    while True:
        self.tour.append(index)
        next_index = prev[index][state]
        if next_index is None:
            break
        next_state = state | (1 << next_index)
        state = next_state
        index = next_index
    self.tour.append(self.START_NODE)
    self.ranSolver = True
  def get_tour(self):
    if not self.ranSolver:
        self.solve()
    return self.tour

  def get_tour_cost(self):
    if not self.ranSolver:
        self.solve()
    return self.minTourCost




**Recursive TSP Method**

The `tsp()` method computes the minimum tour cost using dynamic programming. It checks if all nodes have been visited, returns cached results if available, and iterates through possible next nodes to find the minimum cost path.

**Solve Method**

The `solve()` method initializes the state and memoization structures, calls the recursive tsp() method, and regenerates the path based on the prev array.

**Methods to Get Tour and Tour Cost**

These methods return the optimal tour and its cost. If the solver hasn't been run yet, they call the `solve()` method.

**Example usage**

In [15]:
!pip install pympler

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 3.7 MB/s eta 0:00:00


In [18]:
import time
import pandas as pd
import tracemalloc
from pympler import asizeof

def generate_symmetric_distance_matrix(n, seed=None, min_val=1, max_val=100):
    import random
    if seed is not None:
        random.seed(seed)
    matrix = [[0]*n for _ in range(n)]
    for i in range(n):
        for j in range(i):
            val = random.randint(min_val, max_val)
            matrix[i][j] = matrix[j][i] = val
    return matrix

def test_recursive_tsp(sizes, seed_base=123):
    results = []

    for size in sizes:
        try:
            dist = generate_symmetric_distance_matrix(size, seed=seed_base + size)
            tsp = TspDynamicProgrammingRecursive(dist)
            tracemalloc.start()
            start = time.time()
            tsp.solve()
            duration = time.time() - start
            current, peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()
            results.append({
                "approach": "Recursive",
                "size": size,
                "cost": tsp.get_tour_cost(),
                "tour": tsp.get_tour(),
                "time": duration,
                "mem_peak_kb": peak / 1024,
                "mem_object_kb": asizeof.asizeof(tsp) / 1024
            })
        except Exception as e:
            results.append({
                "approach": "Recursive",
                "size": size,
                "cost": None,
                "tour": None,
                "time": None,
                "error": str(e)
            })

    return pd.DataFrame(results)


In [19]:
df = test_recursive_tsp([4, 5, 6, 7, 8, 9, 10, 11, 12, 13])
df[["approach", "size", "cost", "time", "mem_peak_kb", "mem_object_kb"]]

,approach,size,cost,time,mem_peak_kb,mem_object_kb
0,Recursive,4,103,0.000092,1.304688,1.554688
1,Recursive,5,156,0.000149,2.898438,1.835938
2,Recursive,6,155,0.000291,6.445312,2.195312
3,Recursive,7,269,0.000798,14.851562,2.507812
4,Recursive,8,203,0.002069,33.320312,3.023438
5,Recursive,9,216,0.009082,73.811523,3.242188
6,Recursive,10,223,0.027775,162.172852,3.757812
7,Recursive,11,269,0.091748,387.078125,4.070312
8,Recursive,12,223,0.195784,773.351562,4.585938
9,Recursive,13,168,0.554129,1730.931641,4.992188
